In [ ]:
from __future__ import absolute_import
import numpy as np
import keras
import tensorflow as tf
import random
from keras.layers import Layer, Conv2D, LeakyReLU, InputSpec, Flatten, Dense, Dropout
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
import math

In [ ]:
class FractionalPooling2D(Layer):
	def __init__(self, pool_ratio = None, pseudo_random = False, overlap = False, name ='FractionPooling2D', **kwargs):
		self.pool_ratio = pool_ratio
		self.input_spec = [InputSpec(ndim=4)]
		self.pseudo_random = pseudo_random
		self.overlap = overlap
		self.name = name
		super(FractionalPooling2D, self).__init__(**kwargs)
		
	def call(self, input):
		[batch_tensor,row_pooling,col_pooling] = tf.nn.fractional_max_pool(input, pooling_ratio = self.pool_ratio, pseudo_random = self.pseudo_random, overlapping = self.overlap, seed2 = 1, seed = 1, deterministic=True)
		return(batch_tensor)
		
	def compute_output_shape(self, input_shape):
	
		if(K.image_data_format() == 'channels_last'):
			if(input_shape[0] != None):
				batch_size = int(input_shape[0]/self.pool_ratio[0])
			else:
				batch_size = input_shape[0]
			width = int(input_shape[1]/self.pool_ratio[1])
			height = int(input_shape[2]/self.pool_ratio[2])
			channels = int(input_shape[3]/self.pool_ratio[3])
			return(batch_size, width, height, channels)
			
		elif(K.image_data_format() == 'channels_first'):
			if(input_shape[0] != None):
				batch_size = int(input_shape[0]/self.pool_ratio[0])
			else:
				batch_size = input_shape[0]
			channels = int(input_shape[1]/self.pool_ratio[1])
			width = int(input_shape[2]/self.pool_ratio[2])
			height = int(input_shape[3]/self.pool_ratio[3])
			return(batch_size, channels, width, height)
		
	def get_config(self):
		config = {'pooling_ratio': self.pool_ratio, 'pseudo_random': self.pseudo_random, 'overlap': self.overlap, 'name':self.name}
		base_config = super(FractionalPooling2D, self).get_config()
		return dict(list(base_config.items()) + list(config.items()))
		
	def build(self, input_shape):
		self.input_spec = [InputSpec(shape=input_shape)]

In [ ]:
batch_size = 128
epochs = 100
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=45,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.1,
        # set range for random zoom
        zoom_range=0.1,
        # set range for random channel shifts
        channel_shift_range=0.1,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format="channels_last",#None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)
depth = 7
do = np.linspace(.0, .5, depth)
print(do)

In [ ]:
pr = math.pow(2, 1/3) - 5e-2
print(pr)
resolution = 32.0
model = Sequential()

model.add(Conv2D(2, (2, 2), input_shape=X_train.shape[1:], padding='same'))
model.add(LeakyReLU(alpha = 0.3))
model.add(Dropout(do[0]))
model.add(FractionalPooling2D(pool_ratio=(1, pr, pr, 1),pseudo_random = True, overlap=True))
resolution /= pr
resolution = math.floor(resolution)
for i in range(2, depth-2):
    model.add(Conv2D(2*i, (2, 2), padding='same'))
    model.add(LeakyReLU(alpha = 0.3)) 
    model.add(Dropout(do[i-1]))
    model.add(FractionalPooling2D(pool_ratio=(1, pr, pr, 1),pseudo_random = True, overlap=True))
    resolution /= pr
    resolution = math.floor(resolution)
    print(resolution)

model.add(Conv2D(2*depth-1, (2, 2), padding='same')) # 160*13
model.add(LeakyReLU(alpha = 0.3))
model.add(Dropout(do[-2]))
model.add(Conv2D(2*depth, (1, 1), padding='same')) # 160*14
model.add(LeakyReLU(alpha = 0.3))
model.add(Dropout(do[-1]))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

opt = keras.optimizers.Adadelta(1,decay=1e-4) #0.1
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

In [ ]:
model.fit_generator( datagen.flow(X_train, y_train, batch_size=batch_size), workers=4, validation_data = [X_test,y_test], epochs=epochs, shuffle=True, verbose=1, callbacks=[ReduceLROnPlateau(factor=.5, patience=3, verbose=1)])